 ## Solving Capacitated Lot Sizing Problem using CPLEX's python module DOCPLEX

In [1]:
import random as r
import numpy as np
import time

r.seed(50)

no_of_items = 10
no_of_machines = 2
no_of_period = 4

# setting up the cost
low_setup_time = np.asarray([int(r.uniform(a=10,b=50)) for i in range(no_of_items*no_of_machines)])
high_setup_time = 1.5*low_setup_time

low_setup_cost = np.asarray([int(r.uniform(a=5,b=95)) for i in range(no_of_items*no_of_machines)])
high_setup_cost = 10*low_setup_cost

unit_production_cost = [round(r.uniform(a=1.5,b=2.5), 2) for i in range(no_of_items*no_of_machines)]
unit_inventory_cost = [round(r.uniform(a=0.2,b=0.4),2) for i in range(no_of_items*no_of_period)]
unit_production_time = [int(r.uniform(a=1,b=5)) for i in range(no_of_items*no_of_machines)]

demand = [round(r.uniform(a=1,b=180)) for i in range(no_of_items*no_of_period)]

In [2]:
# Assigning Costs:

Item_list = [i for i in range(1,no_of_items+1)]
Machine_list = [j for j in range(1,no_of_machines+1)]
Period_list = [t for t in range(1,no_of_period+1)]

item_machine_pair = [(i,j) for i in Item_list for j in Machine_list]
item_period_pair = [(i,t) for i in Item_list for t in Period_list]
machine_period_pair = [(j,t) for j in Machine_list for t in Period_list]

Bij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_time)}   # Unit production time of item i on machine j;
Cij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_cost)}   # Unit production cost of item i on machine j;
Hi = {i: s for i,s in zip(Item_list,unit_inventory_cost)}                     # Unit inventory cost of item i per period;
Dit = {(i,t): s for (i,t),s in zip(item_period_pair,demand)}                  # Demand of item i at period t;
Fij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_time)}         # Setup time of item i on machine j;
Sij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_cost)}         # Setup cost of item i on machine j.

In [3]:
# Instanciating the variables:

Vars = [(items,machines,periods) for items in Item_list for machines in Machine_list for periods in Period_list]

In [4]:
from docplex.mp.model import Model

In [5]:
# Initializing the Model:

mip = Model('Capacitated Lot Sizing Problem')

In [6]:
# Declaring decision variables:

X = mip.integer_var_dict(Vars, lb=0, name='x')
Y = mip.binary_var_dict(Vars,name='y')
S = mip.integer_var_dict(item_period_pair, lb=0, name='s')

In [7]:
# Averaging out demand of each item per period in order to calculate capacity of each machine per period:

Cap = mip.sum(((Dit[i,t]/no_of_machines)*Bij[i,j]) + Fij[i,j] for i,j,t in X)                       
capacity_per_period = [int((Cap/(no_of_machines*no_of_period)).constant)]
Qj = {j: s for j,s in zip(Machine_list,capacity_per_period*no_of_period)}           # Capacity of machine j per period;
M = max(capacity_per_period)                                                        # Maximum capacity of machines per period.

In [8]:
cost_list = [Bij,Cij,Hi,Dit,Fij,Sij,Qj,M]

In [9]:
# Constraints:

# constraint_1 ensure that the demand of each item in each period should be satisfied 
constraint_1 = mip.add_constraints(mip.sum(X[i,j,t] for j in range(1,no_of_machines+1)) + (-S[i,t] + S[i,t-1] if t-1 > 0 else -S[i,t]) == Dit[i,t] for i,t in S)

# constraint_2 restrict the production capacity for each machine j at each period t
constraint_2 = mip.add_constraints(mip.sum(Bij[i,j]*X[i,j,t] + Fij[i,j]*Y[i,j,t] for i in range(1,no_of_items+1)) <= Qj[j] for j,t in zip(Machine_list,Period_list))

# constraint_3 impose the setup cost on machine j at period t for item i when Xijt is positive.
constraint_3 = mip.add_constraints(X[i,j,t] <= M*Y[i,j,t] for i,j,t in X)

In [10]:
# Objective function:

setup_cost = mip.sum(Sij[i,j]*Y[i,j,t] for i,j,t in Y) 
production_cost = mip.sum(Cij[i,j]*X[i,j,t] for i,j,t in X)
inventory_cost = mip.sum(Hi[i]*S[i,t] for i,t in S)
total_cost = setup_cost + production_cost + inventory_cost

In [11]:
# Goal:

mip.minimize(total_cost)

In [12]:
solution = mip.solve(log_output=True)
assert solution is not None
value = round(solution.objective_value,2)

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 8610.040000 after 0.02 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve added 20 rows and 0 columns.
MIP Presolve modified 54 coefficients.
Reduced MIP has 132 rows, 200 columns, and 370 nonzeros.
Reduced MIP has 80 binaries, 120 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (0.43 ticks)
Probing fixed 0 vars, tightened 20 bounds.
Probing time = 0.00 sec. (0.15 ticks)
Tried aggregator 2 times.
MIP Presolve modified 30 coefficients.
Aggregator did 1 substitutions.
Reduced MIP has 131 rows, 199 columns, and 368 nonzeros.
Reduced MIP has 80 binaries, 119 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.41 ticks)
Probing time = 0.00 sec. (0.12 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solu

In [13]:
print("The objective function value is: {}".format(value))

The objective function value is: 8156.81


#### *Checking if items produced are actually produced on respective machine*

In [14]:
from ipynb.fs.full.Solution_Checker import checker
checker(solution,X,Y)

'All cases matched.'

#### *Solving with Lagrangian Relaxation*

In [15]:
from ipynb.fs.full.Lagrangian_relaxed_model import Lagrangian_relaxation
output = Lagrangian_relaxation(cost_list,Vars,no_of_items,no_of_machines,no_of_period)

print("\nUpper bound of the problem = {}".format(output.objective_value))

1> new lagrangian iteration:
	 obj=8190.37, m=[1, 1, 1, 1, 1, 1, 1, 1], p=[0, 0, 0, 0, 0, 1.0, 0, 0]
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0], justifier=1
2> new lagrangian iteration:
	 obj=8190.29, m=[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0], p=[0, 0, 0, 0, 0, 1.0000000000019624, 0, 0]
* Lagrangian relaxation succeeds, best_solution=8190.29, penalty=1, #iterations=2

Upper bound of the problem = 8190.290000000001


In [16]:
time.sleep(2)

#### *Solving the classical problem with lower bound solution obtained from Lagrangian relaxed model*

In [17]:
relaxed_solution = mip.read_mip_starts("clsp_-with-_lagrangian_relaxation.mst")

In [18]:
new_solution = mip.solve(log_output=True)
new_obj_val = round(new_solution.objective_value,2)
if new_obj_val == value: print("\nOptimal! and Obejctive value is: {}".format(new_obj_val))

CPXPARAM_Read_DataCheck                          1

Root node processing (before b&c):
  Real time             =    0.03 sec. (0.07 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.03 sec. (0.07 ticks)

Optimal! and Obejctive value is: 8156.81
